In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
# Import stratifiedKFold
from sklearn.model_selection import KFold

In [2]:
from sklearn.preprocessing import StandardScaler
from pandas.api.types import is_numeric_dtype
def scale_data(data):
    scaler = StandardScaler()
    for col in data.columns:
        if is_numeric_dtype(data[col]):
            data[col] = scaler.fit_transform(data[[col]])
    return data

In [3]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
b = [1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9, 12.0]
c = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
d = [object, object, object, object, object, object, object, object, object, object]
data = {'a': a, 'b': b, 'c': c, 'd': d}
df = pd.DataFrame(data)
df = scale_data(df)
df

,a,b,c,d
0,-1.566699,-1.518325,a,<class 'object'>
1,-1.218544,-1.187601,b,<class 'object'>
2,-0.870388,-0.856876,c,<class 'object'>
3,-0.522233,-0.526152,d,<class 'object'>
4,-0.174078,-0.195428,e,<class 'object'>
5,0.174078,0.135296,f,<class 'object'>
6,0.522233,0.466021,g,<class 'object'>
7,0.870388,0.796745,h,<class 'object'>
8,1.218544,1.127469,i,<class 'object'>
9,1.566699,1.758852,j,<class 'object'>


In [4]:
def cross_validate_model(model, X, y):
    cv_strat = KFold(n_splits=10, shuffle=True, random_state=42)
    cv_results = cross_validate(model, X, y, cv=cv_strat, scoring=['neg_root_mean_squared_error'], return_train_score=True, return_estimator=True)
    return cv_results

In [5]:
def average_cv_results(cv_results):
    avg_results = {}
    print(cv_results.keys())
    for key in cv_results.keys():
        if key == 'estimator':
            continue
        avg_results[key] = cv_results[key].mean()
    # Get best estimator according to RMSE
    best_estimator_index = cv_results['test_neg_root_mean_squared_error'].argmax()
    best_estimator = cv_results['estimator'][best_estimator_index]
    return avg_results, best_estimator

In [6]:
def lazy_model(df, target, drop_cols=[]):
    from lazypredict.supervised import LazyRegressor
    X = df.drop(columns=[target] + drop_cols)
    y = df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
    models, predictions = reg.fit(X_train, X_test, y_train, y_test)
    return models

In [7]:
def feature_importance(model, X, y):
    model.fit(X, y)
    if hasattr(model, 'feature_importances_'):
        feature_importances = pd.DataFrame(model.feature_importances_,
                                       index = X.columns,
                                       columns=['importance']).sort_values('importance', ascending=False)
    elif hasattr(model, 'coef_'):
        feature_importances = pd.DataFrame(model.coef_,
                                       index = X.columns,
                                       columns=['importance']).sort_values('importance', ascending=False)
    else:
        print('Model does not have feature importances')
    return feature_importances

In [15]:
# Load the data from pkl
cf = pd.read_pickle('../../data/processed/cf.pkl')
# Target is producto_1_cf
y = cf['producto_1_cf']
# Features are all columns except producto_1_cf
X = cf.drop(columns=['producto_1_cf'])
# Drop all columns which are not numeric
#X = X.select_dtypes(include=['number'])
drop_cols = ['producto_2_cf','lote', 'lote_parental_cf','id_bio', 'f_h_inicio_cf' , 'f_h_fin_cf' , 'id_centr']
X = X.drop(columns=drop_cols)
X.shape, y.shape

((152, 9), (152,))

In [9]:
cf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    152 non-null    object             
 1   orden_encadenado_cf     152 non-null    int64              
 2   lote_parental_cf        152 non-null    object             
 3   id_bio                  152 non-null    object             
 4   f_h_inicio_cf           152 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              152 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         147 non-null    float64            
 7   turb_inicio_cultivo_cf  152 non-null    float64            
 8   turb_fin_cultivo_cf     152 non-null    float64            
 9   viab_final_cultivo_cf   152 non-null    float64            
 10  id_centr                152 non-null    object             
 11  centr_1_turb_cf         148 non-null    float

In [16]:
# Perform cross validation
# Scale
X = scale_data(X)
model = RandomForestRegressor(n_estimators=50, random_state=42)
cv_results = cross_validate_model(model, X, y)
avg_results, best_estimator = average_cv_results(cv_results)
print('test average RMSE:', avg_results['test_neg_root_mean_squared_error'])
print('train average RMSE:', avg_results['train_neg_root_mean_squared_error'])
print('best_estimator test RMSE:', cv_results['test_neg_root_mean_squared_error'].max())
print(feature_importance(best_estimator, X, y))

dict_keys(['fit_time', 'score_time', 'estimator', 'test_neg_root_mean_squared_error', 'train_neg_root_mean_squared_error'])
test average RMSE: -281.71982480810885
train average RMSE: -108.9460756680335
best_estimator test RMSE: -201.67850881638307
                        importance
turbidez_diff_cf              0.22
turb_fin_cultivo_cf           0.15
centr_2_turb_cf               0.15
centr_1_turb_cf               0.11
viab_final_cultivo_cf         0.10
dur_cf                        0.09
vol_ino_util_cf               0.08
turb_inicio_cultivo_cf        0.08
orden_encadenado_cf           0.01


In [17]:

lazy_model(cf, 'producto_1_cf', drop_cols=drop_cols)

100%|██████████| 42/42 [00:00<00:00, 98.38it/s] 

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000017 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 270
[LightGBM] [Info] Number of data points in the train set: 121, number of used features: 8
[LightGBM] [Info] Start training from score 1672.990337
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
ExtraTreesRegressor,0.18,0.42,196.34,0.04
RandomForestRegressor,0.13,0.39,201.66,0.05
BaggingRegressor,0.08,0.35,207.86,0.01
AdaBoostRegressor,0.05,0.34,210.86,0.03
LassoLarsIC,0.03,0.32,212.91,0.00
ElasticNet,0.03,0.32,213.67,0.00
ElasticNetCV,0.03,0.32,213.76,0.02
BayesianRidge,0.02,0.32,213.81,0.00
LarsCV,0.02,0.31,214.17,0.01
